In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
import mnist_loader 
training_data, validation_data, test_data = mnist_loader.load_data_wrapper() 

In [5]:
import network2 
net = network2.Network([784, 30, 10], cost=network2.CrossEntropyCost)

SyntaxError: Missing parentheses in call to 'print'. Did you mean print("Epoch %s training complete" % j)? (network2.py, line 167)

In [ ]:
>>> net.large_weight_initializer()
>>> net.SGD(training_data[:1000], 400, 10, 0.5,
... evaluation_data=test_data, lmbda = 0.1,
... monitor_evaluation_cost=True, monitor_evaluation_accuracy=True,
... monitor_training_cost=True, monitor_training_accuracy=True)